In [13]:
%pwd

'/Users/palessandro/Documents/GitHub/scRNAseq-AAE'

In [14]:
%cd Documents/GitHub/scRNAseq-AAE/

[Errno 2] No such file or directory: 'Documents/GitHub/scRNAseq-AAE/'
/Users/palessandro/Documents/GitHub/scRNAseq-AAE


In [10]:
import scanpy as sc
import numpy as np
import os, sys

os.makedirs('../scRNAseq-AAE/data', exist_ok=True)
os.makedirs('../scRNAseq-AAE/data/proc', exist_ok=True)

In [12]:
os.makedirs('../scRNAseq-AAE/data/pbmc3k/', exist_ok=True)
!wget http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz -O ../scRNAseq-AAE/data/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz
!cd ../scRNAseq-AAE/data/pbmc3k/; tar -xzf pbmc3k_filtered_gene_bc_matrices.tar.gz; rm pbmc3k_filtered_gene_bc_matrices.tar.gz

--2020-09-07 09:20:07--  http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 104.18.0.173, 104.18.1.173
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.0.173|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz [following]
--2020-09-07 09:20:07--  https://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.0.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7621991 (7,3M) [application/x-tar]
Saving to: ‘../scRNAseq-AAE/data/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz’

../scRNAseq-AAE/dat 100%[===================>]   7,27M  4,75MB/s    in 1,5s    

2020-09-07 09:20:09 (4,75 MB/s) - ‘../scRNAseq-AAE/data/pbmc3k/pbmc3k_fi

In [15]:
data_path = '../scRNAseq-AAE/data/pbmc3k/filtered_gene_bc_matrices/hg19'
results_file = '../scRNAseq-AAE/data/proc/pbmc3k.h5ad'
results_file2 = '../scRNAseq-AAE/data/proc/pbmc3k.tfrecord'

In [16]:
gene_up = 2500
percent_mito_up = 0.05
n_pcs = 40
resolution = 1.0

In [17]:
marker_genes = ['IL7R', 'CD79A', 'MS4A1', 'CD8A', 'CD8B', 'LYZ', 'CD14',
                'LGALS3', 'S100A8', 'GNLY', 'NKG7', 'KLRB1',
                'FCGR3A', 'MS4A7', 'FCER1A', 'CST3', 'PPBP']

In [18]:
new_cluster_names = [
    'CD4 T', 'CD14+ Monocytes',
    'B', 'CD8 T',
    'NK', 'FCGR3A+ Monocytes',
    'Dendritic', 'Megakaryocytes']

In [9]:
from keras.utils.vis_utils import model_to_dot

import tensorflow as tf
from tensorflow.keras.utils import plot_model

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import os, sys

from imp import load_source
from models import *
from utils import *

try:
    tf.enable_eager_execution()
except:
    pass

tf.executing_eagerly()
tf.random.set_seed(32)

In [10]:
parameters = {
    'latent_dim': 10,
    'layers_enc_dim': [200, 100],
    'layers_dec_dim': [100, 200],
    'layers_dis_dim': [200, 100, 50, 40, 30],
    'batch_size': 1000,
    'epochs': 30
}

In [11]:
model = AAE1(**parameters)

In [12]:
model.get_parameters()

,Value,Description
batch_size,1000,batch size
epochs,30,number of epochs
alpha,0.1,alpha coeff. in activation function
do_rate,0.1,dropout rate
kernel_initializer,glorot_uniform,kernel initializer of all dense layers
bias_initializer,zeros,bias initializer of all dense layers
l2_weight,None,weight of l2 kernel regularization
l1_weight,None,weight of l1 activity regularization
latent_dim,10,dimension of latent space Z
layer_1_enc_dim,200,dimension of encoder dense layer 1


In [ ]:
os.makedirs('../data', exist_ok=True)
os.makedirs('../data/proc', exist_ok=True)
os.makedirs('../data/pbmc3k/', exist_ok=True)
!wget http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz -O ../data/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz
!cd ../data/pbmc3k/; tar -xzf pbmc3k_filtered_gene_bc_matrices.tar.gz; rm pbmc3k_filtered_gene_bc_matrices.tar.gz

In [ ]:
data_path = '../data/pbmc3k/filtered_gene_bc_matrices/hg19'
results_file = '../data/proc/pbmc3k.h5ad'
results_file2 = '../data/proc/pbmc3k.tfrecord'

In [ ]:
gene_up = 2500
percent_mito_up = 0.05
n_pcs = 40
resolution = 1.0

In [ ]:
marker_genes = ['IL7R', 'CD79A', 'MS4A1', 'CD8A', 'CD8B', 'LYZ', 'CD14',
                'LGALS3', 'S100A8', 'GNLY', 'NKG7', 'KLRB1',
                'FCGR3A', 'MS4A7', 'FCER1A', 'CST3', 'PPBP']

In [ ]:
new_cluster_names = [
    'CD4 T', 'CD14+ Monocytes',
    'B', 'CD8 T',
    'NK', 'FCGR3A+ Monocytes',
    'Dendritic', 'Megakaryocytes']